In [379]:
import uiautomator2 as ui2
import os
import pandas as pd
import time
import unittest
import json
import re
import csv

# 主要函数

In [380]:
# 获取红外分析仪的红码
def level2redcode(code_list, frequency=38000, state=305):
    listValue = []
    listValue1 = []
    if isinstance(code_list, str):
        code_list = eval(code_list)
#     list1 = code_list[2:]  # 去掉频率和长度
    list1 = code_list  # 去掉频率和长度
    if not list1:
        return 38000, []
    for li in range(len(list1)):
        n = round(int(list1[li]) / state)
        if (n > 255):
            listValue1.append(0)
            listValue1.append(n >> 8 & 0xff)
            listValue1.append(n & 0xff)
        else:
            listValue1.append(n)
    q1 = round(int(frequency / 1000), 0)  # 要取小数将第一个int改为float，最后0改要求小数位
    if (q1 > 255):
        listValue.append(q1 >> 8 & 0xff)
        listValue.append(q1 & 0xff)
    else:
        listValue.append(q1)
        listValue.append(0)
    if (len(listValue1) > 255):
        listValue.append(len(listValue1) & 0xff)
        listValue.append(len(listValue1) >> 8 & 0xff)
    else:
        listValue.append(len(listValue1))
        listValue.append(0)
    listValue = listValue + listValue1
    return listValue

def get_redcode():
    fr = open("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt", "r")
    levelcode = fr.read().split("\n")[-2]
    fr.close()
    redcode = level2redcode([int(i) for i in levelcode.split(" ")])
    return redcode

In [159]:
# 获取品牌列表
df = pd.read_excel("./Sensibo空调品牌.xlsx")

In [5]:
# 重启软件
def startApp():
    serial = '1333e1d50606'
    d = ui2.connect_usb(serial)
    # 关闭APP
    d.app_stop("com.sensibo.app")
    # 启动App
    d.app_start("com.sensibo.app")
    time.sleep(8)
    # 点击进入到面板
    d.click(0.53, 0.246)
    return d

In [6]:
d = startApp()

In [381]:
def selectBrand(brand):
    d(text="Tab 4 of 4").click()
    time.sleep(2)
    d.click(0.455, 0.847)
    
    i = 0
    while not d(text="Next").exists:
        time.sleep(1)
        i += 1
        if i > 60:
            break
    d(text="Next").click()
    
    i = 0
    while not d(text="Set remote manually").exists:
        time.sleep(1)
        i += 1
        if i > 60:
            break
    
    d(text="Set remote manually").click()
    
    i = 0
    while not d(text="Send a test On/Off command").exists:
        time.sleep(1)
        i += 1
        if i > 60:
            break
            
    d.click(0.107, 0.396)
    time.sleep(1)
    # 点开品牌列表
    d.click(0.697, 0.468)
    time.sleep(1)
    while not d(text=brand).exists:
        d.swipe(0.5, 0.145, 0.5, 0.084)

    # 选择品牌
    d(text=brand).click()

In [339]:
selectBrand("Omega")

In [357]:
d(text="Air-Con").exists

True

In [382]:
def getBrandModelList():
    # 获取该品牌红码型号
    d.click(0.539, 0.533)
    model_list = []
    st = None
    while True:
        for i in d.xpath('//android.view.View').all():
            if i.text == "" or i.text in model_list:
                continue
            else:
                model_list.append(i.text)
        if st == len(model_list):
            break
        st = len(model_list)
        d.swipe(0.5, 0.15, 0.5, 0.084)
    # 退回界面
    d.click(0.141, 0.624)
    return model_list

In [80]:
model_list = getBrandModelList()

In [81]:
model_list

['Lively Walrus']

In [383]:
def getBrandRedcode(brand, model):
    # 选择红码面板
    if not d(text="Send a test On/Off command").exists:
        selectBrand(brand)
    d.click(0.539, 0.533)
    time.sleep(1)
    while not d(text=model).exists:
        print(1)
        d.swipe(0.5, 0.15, 0.5, 0.084)
    d(text=model).click()
    time.sleep(1)
    d(text="Done").click()
    
    i = 0
    while not d(text="Tab 1 of 4").exists:
        time.sleep(1)
        i += 1
        if i > 60:
            break
            
    d(text="Tab 1 of 4").click()
    return True

In [83]:
getBrandRedcode("Aux", "Lively Walrus")

True

In [384]:
temp_map = {
    61: 16,
    63: 17,
    64: 18,
    66: 19,
    68: 20,
    70: 21,
    72: 22,
    73: 23,
    75: 24,
    77: 25,
    79: 26,
    81: 27,
    82: 28,
    84: 29,
    86: 30,
    88: 31,
    90: 32
}

In [385]:
# 获取当前面板状态
def get_now_status():
    status = d.xpath('//android.view.View').all()
    index = 0
    while True:
        if status[index].text == "Living room":
            index += 1
            break
        index += 1
    now_status = status[index].text
    if "." in now_status:
        now_status = status[index-2].text
    now_status = now_status.split(" ")
    if len(now_status) >= 3:
        now_status[2] = int(now_status[2].replace("°", ""))
        if now_status[2] > 50:
            if now_status[2] in temp_map:
                now_status[2] = temp_map[now_status[2]]
            elif now_status[2] < 61:
                now_status[2] = 10
            elif now_status[2] > 90:
                now_status[2] = 33
#     now_status[2] = round((now_status[2] - 32) * 5 / 9)
    return now_status
get_now_status()

['Cool', 'high', 20]

In [386]:
# 获取中间温度, 只有中间温度与头状态温度一直,才发完红码
def getMiddleTemperture():
    i = 0
    status = d.xpath('//android.view.View').all()
    temp = 0
    while i < len(status):
        if status[i].text == "°":
            if status[i-1].text != "--":
                temp = int(status[i-1].text)
            break
        i+=1
    if temp > 50:
        if temp in temp_map:
            temp = temp_map[temp]
        elif temp < 61:
            temp = 10
        elif temp > 90:
            temp = 33
#     temp = round((temp - 32) * 5 / 9)
    return temp
getMiddleTemperture()

20

In [387]:
def getRedcode():
    # 获取当前面板红码
    rel_data = []
    # 获取关机状态
    if not d(text="TURN OFF").exists:
        d(text="TURN ON").click()
    if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
        os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
    
    i = 0
    while not d(text="TURN OFF").exists():
        time.sleep(1)
        i += 1
        if i > 60:
            break

    d(text="TURN OFF").click()
    i = 0
    while not d(text="TURN ON").exists:
        time.sleep(1)
        i += 1
        if i > 60:
            break
            
    redcode=str(get_redcode()) 
    #  status,mode,speed,up_down_direct,temperature,redcode
    rel_data.append(['close','cool','auto','auto', 27, redcode])
    # 返回开机状态
    d(text="TURN ON").click()

    i = 0
    while not d(text="TURN OFF").exists:
        time.sleep(1)
        i += 1
        if i > 60:
            break
    
#     speed_list = {'Quiet': "speed1", 'Low': "speed2", 'Medium':"speed3", 'High':"speed4", 'Auto': "auto", 'Strong': "speed5"}
#     direct_list = {"Stopped (auto)":"close", 
#                    "Moving": "auto", 
#                    "Fixed top": "fix1", 
#                    "Fixed middle top": "fix2", 
#                    "Fixed middle": "fix3", 
#                    "Fixed middle bottom": "fix4", 
#                    "Fixed bottom": "fix5"}
    speed_list = {'Low': "low", 'Medium':"middle", 'High':"high", 'Auto': "auto"}
    direct_list = {"Stopped (auto)":"fix",  "Moving": "auto"}
    
#     mode_list = {'Cool': "cool", 'Heat': "heat", 'Fan': "wind", 'Dry': "humidity", 'Auto': "auto"}
    mode_list = {'Cool': "cool", 'Heat': "heat"}
    for mode in mode_list:
        print(f"mode: {mode}")
        d.click(0.182, 0.660)
        time.sleep(2)
        if d(text=mode).exists:
            d(text=mode).click()
            
            status = get_now_status()
            
            i = 0
            while status[0].lower() != mode.lower():
                time.sleep(1)
                status = get_now_status()
                i += 1
                if i > 60:
                    break

            status_temp = True
            status_wind = True
            if len(status) < 3:
                status_temp = False
                print("该模式 {} 控制不了温度".format(mode))
            if len(status) < 2:
                status_temp = False
                print("该模式 {} 控制不了风速".format(mode))
            # 判断风向
            status_direct = True
            # 先判断有没有风向
            d.click(0.808, 0.660)
            time.sleep(3)
            if d(text="When temperature goes above").exists:
                print("该面板没有风向")
                status_direct = False
            # 退回面板
            d(resourceId="com.android.systemui:id/back").click()
            
            if status_temp and status_wind and status_direct:
                print('state: 1')
                for speed in speed_list:
                    d.click(0.508, 0.660)
                    time.sleep(1)
                    if d(text=speed).exists:
                        d(text=speed).click()
                        i = 0
                        while get_now_status()[1].lower() != speed.lower():
                            time.sleep(1)
                            i += 1
                            if i > 60:
                                break
                                
                        for direct in direct_list:
                            print(f"mode: {mode}, speed: {speed}, direct: {direct}")
                            # 开始选择风向
                            d.click(0.808, 0.660)
                            time.sleep(1)
                            if d(text=direct).exists:
                                d(text=direct).click()
                                # 获取温度红码
                                
                                i = 0
                                while d(text=direct).exists:
                                    print(f"direct: {direct}")
                                    time.sleep(1)
                                    i += 1
                                    if i > 60:
                                        break
                                temp_dic = getTempDic()
                                while len(temp_dic) < 12:
                                    temp_dic = getTempDic(temp_dic)
                                for temp in temp_dic:
                                    rel_data.append(['open', mode_list[mode], speed_list[speed], direct_list[direct], temp, temp_dic[temp]])
                            else:
                                print("没有该风向, {}".format(direct))
                                # 退回面板
                                d(resourceId="com.android.systemui:id/back").click()
                                time.sleep(1)
                    else:
                        print("没有该风速: {}".format(speed))
                        d(resourceId="com.android.systemui:id/back").click()
                        time.sleep(1)
                        continue
            elif status_temp and status_wind and not status_direct:
                print('state: 2')
                for speed in speed_list:
                    d.click(0.508, 0.660)
                    time.sleep(1)
                    if d(text=speed).exists:
                        print(f"mode: {mode}, speed: {speed}")
                        d(text=speed).click()
                        
                        i = 0
                        while get_now_status()[1].lower() != speed.lower():
                            time.sleep(1)
                            i += 1
                            if i > 60:
                                break
                        # 获取温度红码
                        temp_dic = getTempDic()
                        while len(temp_dic) < 12:
                            temp_dic = getTempDic(temp_dic)
                        for temp in temp_dic:
                            rel_data.append(['open', mode_list[mode], speed_list[speed], 'auto', temp, temp_dic[temp]])
                    else:
                        print("没有该风速: {}".format(speed))
                        d(resourceId="com.android.systemui:id/back").click()
                        time.sleep(1)
                        continue
            elif status_temp and not status_wind and status_direct:
                print('state: 3')
                for direct in direct_list:
                    # 开始选择风向
                    d.click(0.808, 0.660)
                    time.sleep(1)
                    if d(text=direct).exists:
                        print(f"mode: {mode}, direct: {direct}")
                        d(text=direct).click()
                        
                        i = 0
                        while d(text=direct).exists:
                            print(f"direct: {direct}")
                            time.sleep(1)
                            i += 1
                            if i > 60:
                                break
                        # 获取温度红码
                        temp_dic = getTempDic()
                        while len(temp_dic) < 12:
                            temp_dic = getTempDic(temp_dic)
                        for temp in temp_dic:
                            rel_data.append(['open', mode_list[mode], '0', direct_list[direct], temp, temp_dic[temp]])
                    else:
                        print("没有该风向, {}".format(direct))
                        # 退回面板
                        d(resourceId="com.android.systemui:id/back").click()
                        time.sleep(1)
            elif not status_temp and status_wind and status_direct:
                print('state: 4')
                for speed in speed_list:
                    d.click(0.508, 0.660)
                    time.sleep(1)
                    if d(text=speed).exists:
                        d(text=speed).click()
                        
                        i = 0
                        while get_now_status()[1].lower() != speed.lower():
                            time.sleep(1)
                            i += 1
                            if i > 60:
                                break
                                
                        for direct in direct_list:
                            print(f"mode: {mode}, speed: {speed}, direct: {direct}")
                            # 开始选择风向
                            d.click(0.808, 0.660)
                            time.sleep(1)
                            if d(text=direct).exists:
                                d(text=direct).click()
                                
                                i = 0
                                while d(text=direct).exists:
                                    print(f"direct: {direct}")
                                    time.sleep(1)
                                    i += 1
                                    if i > 60:
                                        break
                                # 控制不了温度的,通过开机键,获取红码
                                d(text="TURN OFF").click()
                                
                                i = 0
                                while not d(text="TURN ON").exists:
                                    time.sleep(1)
                                    i += 1
                                    if i > 60:
                                        break
                                if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                                    os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                                d(text="TURN ON").click()
                                
                                i = 0
                                while not d(text="TURN OFF").exists:
                                    time.sleep(1)
                                    i += 1
                                    if i > 60:
                                        break
                                        
                                redcode = str(get_redcode())
                                rel_data.append(['open', mode_list[mode], speed_list[speed], direct_list[direct], 0, redcode])
                            else:
                                print("没有该风向, {}".format(direct))
                                # 退回面板
                                d(resourceId="com.android.systemui:id/back").click()
                                time.sleep(1)
                    else:
                        print("没有该风速: {}".format(speed))
                        d(resourceId="com.android.systemui:id/back").click()
                        time.sleep(1)
                        continue
            elif status_temp and not status_wind and not status_direct:
                print('state: 5')
                # 获取温度红码
                temp_dic = getTempDic()
                while len(temp_dic) < 12:
                    temp_dic = getTempDic(temp_dic)
                for temp in temp_dic:
                    rel_data.append(['open', mode_list[mode], '0', 'auto', temp, temp_dic[temp]])
            elif not status_temp and status_wind and not status_direct:
                print('state: 6')
                for speed in speed_list:
                    d.click(0.508, 0.660)
                    time.sleep(1)
                    if d(text=speed).exists:
                        print(f"mode: {mode}, speed: {speed}")
                        d(text=speed).click()
                        
                        i = 0
                        while get_now_status()[1].lower() != speed.lower():
                            time.sleep(1)
                            i += 1
                            if i > 60:
                                break

                        # 控制不了温度的,通过开机键,获取红码
                        d(text="TURN OFF").click()
                        
                        i = 0
                        while not d(text="TURN ON").exists:
                            time.sleep(1)
                            i += 1
                            if i > 60:
                                break
                        if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                            os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                        d(text="TURN ON").click()
                        
                        i = 0
                        while not d(text="TURN OFF").exists:
                            time.sleep(1)
                            i += 1
                            if i > 60:
                                break
                        redcode = str(get_redcode())
                        rel_data.append(['open', mode_list[mode], speed_list[speed], 'auto', 0, redcode])
                    else:
                        print("没有该风速: {}".format(speed))
                        d(resourceId="com.android.systemui:id/back").click()
                        time.sleep(1)
                        continue
        else:
            print("没有该模式: {}".format(mode))
            d(resourceId="com.android.systemui:id/back").click()
            time.sleep(1)
    df = pd.DataFrame(rel_data, columns=['state', 'mode', 'speed', 'direct', 'temperature', 'redcode'])
    return df

In [388]:
# 获取温度红码
def getTempDic(temp_dic = {}):
    # 开始温度遍历
    min_temp = get_now_status()[2]
    # 退到最低温度
    while True:
        d.click(0.119, 0.441)
        middle_temp = getMiddleTemperture()
        i = 0
        while middle_temp != get_now_status()[2]:
            time.sleep(1)
            i += 1
            if i > 60:
                break
            
        if middle_temp == min_temp:
            break
        else:
            min_temp = middle_temp
            if min_temp < 16:
                break
    if min_temp >= 16:
        # 加一度
        d.click(0.88, 0.441)
        print(min_temp)
        i = 0
        while get_now_status()[2] != min_temp +1:
            time.sleep(1)
            i += 1
            if i > 60:
                break
                
        if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
            os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
        # 减一度
        d.click(0.119, 0.441)
        
        i = 0
        while get_now_status()[2] != min_temp:
            time.sleep(1)
            i += 1
            if i > 60:
                break
        redcode=str(get_redcode())
        temp_dic[min_temp] = redcode

    max_temp = None
    while True:
        if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
            os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
        d.click(0.88, 0.441)
        now_temp = getMiddleTemperture()
        if now_temp == max_temp:
            break
        max_temp = now_temp
        
        i = 0
        while get_now_status()[2] != now_temp:
            time.sleep(1)
            i += 1
            if i > 60:
                break
                
        print(now_temp)
        redcode=str(get_redcode())
        temp_dic[now_temp] = redcode
        if max_temp == 32:
            break
    if len(temp_dic) < 13:
        temp_dic = getTempDic(temp_dic)
    return temp_dic

In [187]:
temp_dic = getTempDic()

In [188]:
temp_dic

{27: '[38, 0, 72, 0, 0, 1, 37, 141, 21, 14, 19, 15, 21, 14, 19, 15, 19, 15, 21, 50, 19, 15, 21, 14, 19, 52, 19, 51, 19, 51, 20, 51, 20, 51, 22, 14, 21, 50, 19, 51, 21, 50, 21, 14, 19, 15, 21, 14, 19, 51, 20, 15, 19, 15, 21, 14, 19, 15, 19, 51, 20, 51, 22, 49, 22, 14, 19, 51, 22, 49, 20, 51, 20, 0, 12, 203]'}

# main

In [372]:
success = set()

In [398]:
startApp()
for brand in list(df[(df['device'].isnull()) | (df['device'] == "None")]['brand']):
    if brand in success_brand:
        continue
    print(brand)
    selectBrand(brand)
    model_list = getBrandModelList()
    for model in model_list:
        if (brand, model) in success:
            continue
        i = 1
        while True:
            if i > 10:
                break
            try:
                print((brand, model))
                getBrandRedcode(brand, model)
                df_t = getRedcode()
                df_t.to_csv("./data/{}_{}.csv".format(brand.replace("/"," "), model), index=False)
                success.add((brand, model))
                print("success")
                break
            except Exception as e:
                i += 1
                print("error")
                print(e)
                startApp()
                print("*" * 50)
    success_brand.add(brand)

Artel
('Artel', 'Adorable Grouse')
error
[Errno 2] No such file or directory: 'C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt'
**************************************************
('Artel', 'Adorable Grouse')
error
-32002 Client error: <> data: Selector [text='Tab 4 of 4'], method: None
**************************************************
('Artel', 'Adorable Grouse')
error
-32002 Client error: <> data: Selector [text='Tab 4 of 4'], method: None
**************************************************
('Artel', 'Adorable Grouse')
error
-32002 Client error: <> data: Selector [text='Tab 4 of 4'], method: None
**************************************************
('Artel', 'Adorable Grouse')
error
-32002 Client error: <> data: Selector [text='Tab 4 of 4'], method: None


KeyboardInterrupt: 

In [389]:
success

set()

In [348]:
success

{('Akai', 'Otherworldly Crane'),
 ('Aux', 'Lively Walrus'),
 ('Dyson', 'Dashing Panda'),
 ('Dyson', 'Dashing Sheep'),
 ('Dyson', 'Immense Albatross'),
 ('Dyson', 'Perpetual Chipmunk'),
 ('GE', 'Elusive Cat'),
 ('GE', 'Gigantic Sandpiper'),
 ('GE', 'Majestic Hamster'),
 ('GE', 'Masked Grasshopper'),
 ('GE', 'Solid Falcon'),
 ('GE', 'Witty Kestrel'),
 ('Honeywell', 'Intuitive Raven'),
 ('Honeywell', 'Recurring Camel'),
 ('Lasko', 'Fantastic Turtle'),
 ('Omega', 'Dynamic Vampire Bat'),
 ('Platinum', 'Dashing Eel'),
 ('Royal', 'Attentive Walrus'),
 ('Royal', 'Benevolent Gorilla'),
 ('Royal', 'Inventive Cat'),
 ('Royal', 'Symbolic Lark')}

In [283]:
# success = set()
# success.add(('Aux', 'Lively Walrus'))
# success.add(('Honeywell', 'Intuitive Raven'))

In [356]:
d(text='Air-Con').exists

True